In [1]:
## import modules here
from pyspark import SparkContext, SparkConf
from time import time
import pickle


def createSC():
    conf = SparkConf()
    conf.setMaster("local[*]")
    conf.setAppName("C2LSH")
    sc = SparkContext(conf = conf)
    return sc

with open("toy/toy_hashed_data", "rb") as file:
    data = pickle.load(file)

with open("toy/toy_hashed_query", "rb") as file:
    query_hashes = pickle.load(file)

alpha_m = 10
beta_n = 10

sc = createSC()
data_hashes = sc.parallelize([(index, x) for index, x in enumerate(data)])


In [ ]:
def c2lsh(data_hashes, query_hashes, alpha_m, beta_n):

    offset, cand = 0, 0

#     query_hashes.cache()
    cand_set = set()
    length = len(query_hashes)
    
    while True:
#         for t, v in data_hashes.groupByKey().collect():
            
#             if count(v, query_hashes, offset, length) >= alpha_m:
#                 cand_set.add(t)
        z = data_hashes.map(count(query_hashes, offset, length))        
        if len(cand_set) < beta_n:
            offset += 1
        else:
            break
    
    rdd = sc.parallelize(cand_set)
    return rdd

res = c2lsh(data_hashes, query_hashes, alpha_m, beta_n).collect()

print(res)

In [ ]:
def count(data_hash, query_hashes, offset, length):
    count = 0

    for i in range(length):
        if abs(data_hash.data[0][i] - query_hashes[i]) <= offset:
            count += 1
    
    return count
    
    

In [ ]:
def c(data_hash, query_hashes, offset, length, alpha_m):
    count = 0
    k, v = data_hash
    
    for i in range(length):
        if abs(v[i] - query_hashes[i]) <= offset:
            count += 1
    
    if count >= alpha_m:
        return k


In [ ]:
d = data_hashes.map(lambda x: c(x, query_hashes, 0, 32, alpha_m))
a = d.filter(lambda x: x[1] >= alpha_m)

a

In [ ]:
d = d.subtractByKey(a).collect()
d

In [ ]:
def c2lsh(data_hashes, query_hashes, alpha_m, beta_n):

    offset, cand = 0, 0

#     query_hashes.cache()
#     cand_set = set()
    length = len(query_hashes)
    a = None
    isFirst = True
    
    while True:
#         for t, v in data_hashes.groupByKey().collect():
            
#             if count(v, query_hashes, offset, length) >= alpha_m:
#                 cand_set.add(t)
        if isFirst:
            d = data_hashes.map(lambda x: c(x, query_hashes, offset, length, alpha_m))
            a = d.filter(lambda x: x is not None)
            isFirst = False
            print(a.collect())
        else:
            d = data_hashes.map(lambda x: c(x, query_hashes, offset, length, alpha_m))
            d = d.filter(lambda x: x is not None)
            a = a.union(d)
            print(type(a))
        
        if a.count() < beta_n:
            offset += 1
            data_hashes = data_hashes.subtract(a.map(lambda x: (x, 1)))
        else:
            break
    
#     rdd = sc.parallelize(cand_set)
    return a

res = c2lsh(data_hashes, query_hashes, alpha_m, beta_n).collect()

print(res)

In [28]:
sc.stop()

In [ ]:
length = len(query_hashes)
offset = 0
q = sc.emptyRDD()
d = data_hashes.map(lambda x: c(x, query_hashes, offset, length, alpha_m))
q = sc.union([q, d.filter(lambda x: x[1] >= alpha_m)])
q.collect()

In [9]:
data_hashes = sc.parallelize([(index, x) for index, x in enumerate(data)])

In [40]:
a = sc.parallelize([0, 4])

In [41]:
data_hashes = data_hashes.subtractByKey(a.map(lambda x: (x, None)))
data_hashes.collect()

[(8,
  [40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40]),
 (16,
  [40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40]),
 (24,
  [40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40]),
 (32,
  [40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40]),
 (40,
  [1,
   1,
   1,
   1,
   1,
   1,
   2,
   3,
   4,
   5,
   40,


In [42]:
sc.stop()

In [8]:
d1 = sc.parallelize([1,2,3,4])
d1 = d1.map(lambda x: [x, 5, 5])
d1.collect()
# d1.map(lambda x: (x, index) for index in range(0, 4))

[[1, 5, 5], [2, 5, 5], [3, 5, 5], [4, 5, 5]]

In [3]:
d1.collect()

[1, 2, 3, 4]

In [5]:
z = d1.filter(lambda x: x < 3).count()
z

2

In [18]:
data_hashes = sc.parallelize([(index, x) for index, x in enumerate(data)])
offset = 0
length = len(query_hashes)

In [16]:
def sub(data_hash, query_hashes, offset, length, alpha_m):
    count = 0
    k, v = data_hash
    
    for i in range(length):
        v[i] = abs(v[i] - query_hashes[i])
    
    return k, v

In [24]:
t = data_hashes.map(lambda x: sub(x, query_hashes, offset, length, alpha_m))

t.collect()

[(0,
  [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20]),
 (1,
  [39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20]),
 (2,
  [39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20]),
 (3,
  [39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20,
   20]),
 (4,
  [39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   39,
   20,
   2

In [28]:
j = t.map(lambda x: (x[0], sum(x[1])))
j.collect()


[(0, 440),
 (1, 830),
 (2, 830),
 (3, 830),
 (4, 830),
 (5, 830),
 (6, 830),
 (7, 830),
 (8, 830),
 (9, 830),
 (10, 441),
 (11, 830),
 (12, 830),
 (13, 830),
 (14, 830),
 (15, 830),
 (16, 830),
 (17, 830),
 (18, 830),
 (19, 830),
 (20, 443),
 (21, 830),
 (22, 830),
 (23, 830),
 (24, 830),
 (25, 830),
 (26, 830),
 (27, 830),
 (28, 830),
 (29, 830),
 (30, 446),
 (31, 830),
 (32, 830),
 (33, 830),
 (34, 830),
 (35, 830),
 (36, 830),
 (37, 830),
 (38, 830),
 (39, 830),
 (40, 450),
 (41, 830),
 (42, 830),
 (43, 830),
 (44, 830),
 (45, 830),
 (46, 830),
 (47, 830),
 (48, 830),
 (49, 830),
 (50, 455),
 (51, 830),
 (52, 830),
 (53, 830),
 (54, 830),
 (55, 830),
 (56, 830),
 (57, 830),
 (58, 830),
 (59, 830),
 (60, 461),
 (61, 830),
 (62, 830),
 (63, 830),
 (64, 830),
 (65, 830),
 (66, 830),
 (67, 830),
 (68, 830),
 (69, 830),
 (70, 468),
 (71, 830),
 (72, 830),
 (73, 830),
 (74, 830),
 (75, 830),
 (76, 830),
 (77, 830),
 (78, 830),
 (79, 830),
 (80, 476),
 (81, 830),
 (82, 830),
 (83, 830),
 (

In [22]:
z = t.filter(lambda x: (x[0], z) < offset)

In [23]:
z.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 12.0 failed 1 times, most recent failure: Lost task 1.0 in stage 12.0 (TID 49, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/yash/anaconda3/envs/bd/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/Users/yash/anaconda3/envs/bd/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/yash/anaconda3/envs/bd/lib/python3.6/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 400, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/Users/yash/anaconda3/envs/bd/lib/python3.6/site-packages/pyspark/p

In [49]:
k = t.map(lambda x: (x[0], list(filter(lambda y: y <= offset, x[1]))))
k = k.map(lambda x: (x[0], len(x[1])))
k = k.filter(lambda x: x[1] >= alpha_m)
k.collect()

[(0, 10)]

In [50]:
data_hashes.count()

100

In [52]:
data_hashes = data_hashes.subtractByKey(k)
data_hashes.collect()

[(24,
  [40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40]),
 (48,
  [40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40]),
 (72,
  [40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40]),
 (96,
  [40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40]),
 (12,
  [40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   40,
   

In [37]:
t.map(lambda x: print(type(x[0]), type(x[1])))

PythonRDD[33] at RDD at PythonRDD.scala:53

In [38]:
type(t.)

TypeError: 'PipelinedRDD' object does not support indexing